<center>

<h1><b>Predict hierarchical levels in Knowledge Organization Systems (KOS) </b></h1>
<h2><b>Ahsan Ali </b></h2>
<h2><b>Supervisor: Péter Kiraly</b></h1>
</center>

### It was observed that there are some auxilary tables in KSS apart from main divisions. Due to which many resolveable terms are still not resolved by previous API. After contating the API owner and getting to know more about the API it is observed that there is another end-point which can resolve the terms that are in auxilary tables.

In [1]:
import pandas as pd
import requests

### Loading the previously combined data

In [2]:
df= pd.read_csv("combined_output.csv")
df

,term,Response Code,Returned List,count
0,0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0
1,0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0
2,0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0
3,0,200.0,[],12.0
4,00,NaN,NaN,1.0
...,...,...,...,...
51185,zot,200.0,[],1.0
51186,zs95f,200.0,[],1.0
51187,zur,200.0,[],1.0
51188,zw315m,200.0,[],1.0


### Filtering the terms that were resolved in previous API 

In [3]:
df_invalid = df[df['Returned List'].apply(lambda x: str(x) == "[]" or pd.isna(x))]
df_invalid

,term,Response Code,Returned List,count
3,0,200.0,[],12.0
4,00,NaN,NaN,1.0
5,000,NaN,NaN,28.0
6,000.00,NaN,NaN,1.0
7,0001,NaN,NaN,4.0
...,...,...,...,...
51185,zot,200.0,[],1.0
51186,zs95f,200.0,[],1.0
51187,zur,200.0,[],1.0
51188,zw315m,200.0,[],1.0


In [4]:

def get_result_from_another_endpoint(notation):
    # Define the URL
    url = "https://coli-conc.gbv.de/coli-ana/app//analyze?notation="+str(notation)

    # Send a GET request to the API
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Print the response JSON if the request was successful
        print("Response JSON:", response.json())
        return response.json()
    else:
        # Print an error message if the request was not successful
        print(f"Request failed with status code {response.status_code}")
        return response.status_code


In [5]:
# comenting out this code because it calls the API for thousands of times and saves data

# df_invalid["Returned from second API"]=  df_invalid["term"].apply(get_result_from_another_endpoint)
# df_invalid.to_csv("result_from_new_API.csv")

# instead I am loading frm the saved file
df_invalid= pd.read_csv("result_from_new_API.csv")

In [6]:
df_invalid

,Unnamed: 0,term,Response Code,Returned List,count,Returned from second API
0,3,0,200.0,[],12.0,[{'uri': 'http://dewey.info/class/000.00/e23/'...
1,4,00,NaN,NaN,1.0,[]\n
2,5,000,NaN,NaN,28.0,[]\n
3,6,000.00,NaN,NaN,1.0,[]\n
4,7,0001,NaN,NaN,4.0,[]\n
...,...,...,...,...,...,...
40460,51185,zot,200.0,[],1.0,NaN
40461,51186,zs95f,200.0,[],1.0,NaN
40462,51187,zur,200.0,[],1.0,NaN
40463,51188,zw315m,200.0,[],1.0,NaN


### Merging the result recieved from the new API with the previous data to analyze the difference

In [8]:
df_merged = df.merge(df_invalid[['term', 'Returned from second API']], on='term', how='left')
df_merged.to_csv("combined_output_with_new_API.csv")